# Segmenting and Clustering Neighborhoods in Toronto: Dataframe Creation 

#### First, I import all the libraries I will need in the assignment

In [2]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


bran

## 1. Creation of the Toronto dataframe


In [3]:
# import the library we use to open URLs
import requests

# import the BeautifulSoup library 
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
!pip install lxml


# specify and open the url using urllib.request and put the HTML into the page variable
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

# parse the HTML from our URL 
soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))

neighborhood=pd.DataFrame(df[0])

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |   py36h9f0ad1d_0         160 KB  conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.7.1-py36_1 --> 4.9.0-py36h9f0ad1d_0 conda-forge


beautifulsoup4-4.9.0 | 160 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [4]:
neighborhood.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
neighborhood.tail()

,Postal Code,Borough,Neighborhood
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,NaN


In [6]:
#Only process the cells that have an assigned borough

neighborhood=neighborhood[neighborhood.Borough != 'Not assigned']

neighborhood.tail()

,Postal Code,Borough,Neighborhood
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [7]:
# combine more than one neighborhood when they exist in the same  postal code area using COMMA 

neighborhood['Neighborhood'].replace(r' \/ ', ', ', regex=True, inplace=True)

neighborhood.tail()

,Postal Code,Borough,Neighborhood
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [8]:
#assign borough to neighborhood when neighborhood is Not assigned neighborhood

neighborhood['Neighborhood']=neighborhood['Neighborhood'].replace('Not assigned', neighborhood['Borough'])


In [9]:
neighborhood.shape

(103, 3)

## 2. Get the latitude and the longitude coordinates of each neighborhood

In [10]:
!wget -q -O 'toronto_geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [11]:
torontogeo= pd.read_csv('toronto_geospatial_data.csv')
torontogeo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [12]:
toronto_df = neighborhood.merge(torontogeo)
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## 3. Explore and cluster the Neighborhoods in Etobicoke Borough

In [13]:
#create a dataframe of Etobicoke borough 

Etobicoke_df = toronto_df[toronto_df['Borough'] == 'Etobicoke'].reset_index(drop=True)
Etobicoke_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
5,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
6,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
7,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
8,M9W,Etobicoke,Northwest,43.706748,-79.594054
9,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


In [14]:
#geographical coordinates for Etobicoke

address = 'Etobicoke, TO'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.671459150000004, -79.55249206611668.


In [15]:
# create map of Etobicoke using latitude and longitude values
map_Etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Etobicoke_df['Latitude'], Etobicoke_df['Longitude'], Etobicoke_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Etobicoke)  
    
map_Etobicoke

In [16]:
#define Foursquare details 


CLIENT_ID = 'IB43C4M5C1VPP0XRNG2SSB2R1VQUWVYWODFN1114FWQVLR5Q' 
CLIENT_SECRET = 'N3NWM2XUOXRZDJOPYAY13XA5REUKXSRDEX1TCF23GD5VQJ0O' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IB43C4M5C1VPP0XRNG2SSB2R1VQUWVYWODFN1114FWQVLR5Q
CLIENT_SECRET:N3NWM2XUOXRZDJOPYAY13XA5REUKXSRDEX1TCF23GD5VQJ0O


In [49]:
#Function to explore the neighborhoods venues

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [50]:
#create a dataframe with all Etobicoke venues by calling the function getNearbyVenues

Etobicoke_venues = getNearbyVenues(names=Etobicoke_df['Neighborhood'],latitudes=Etobicoke_df['Latitude'], longitudes=Etobicoke_df['Longitude'])

print(Etobicoke_venues.shape)
Etobicoke_venues.head()

Islington Avenue
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
New Toronto, Mimico South, Humber Bay Shores
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Alderwood, Long Branch
Northwest
The Kingsway, Montgomery Road, Old Mill North
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West
(75, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,Seaforth Golf Club,43.651183,-79.556107,Golf Course
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,LCBO,43.642099,-79.576592,Liquor Store
2,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Starbucks,43.641312,-79.576924,Coffee Shop
3,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,The Beer Store,43.641313,-79.576925,Beer Store
4,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Shoppers Drug Mart,43.641312,-79.576924,Cosmetics Shop


In [51]:
# check the venues returned for each neighborhood 

Etobicoke_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Long Branch",10,10,10,10,10,10
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",7,7,7,7,7,7
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",4,4,4,4,4,4
"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West",13,13,13,13,13,13
"New Toronto, Mimico South, Humber Bay Shores",15,15,15,15,15,15
Northwest,2,2,2,2,2,2
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",3,3,3,3,3,3
"South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens",11,11,11,11,11,11
"The Kingsway, Montgomery Road, Old Mill North",2,2,2,2,2,2


#### Analyze each neighborhood

In [52]:
# one hot encoding
Etobicoke_onehot = pd.get_dummies(Etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Etobicoke_onehot['Neighborhood'] = Etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Etobicoke_onehot.columns[-1]] + list(Etobicoke_onehot.columns[:-1])
Etobicoke_onehot = Etobicoke_onehot[fixed_columns]

Etobicoke_onehot.head()
Etobicoke_onehot.shape

(75, 45)

In [53]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

Etobicoke_grouped = Etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
Etobicoke_grouped
Etobicoke_grouped.shape


(11, 45)

In [54]:
#5 most common venues for each neighborhood 

num_top_venues = 5

for hood in Etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Etobicoke_grouped[Etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   0.2
1             Gym   0.1
2    Dance Studio   0.1
3    Skating Rink   0.1
4  Sandwich Place   0.1


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
            venue  freq
0    Liquor Store  0.14
1      Beer Store  0.14
2  Shopping Plaza  0.14
3            Café  0.14
4     Coffee Shop  0.14


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                 venue  freq
0    Mobile Phone Shop  0.25
1             Bus Line  0.25
2       Sandwich Place  0.25
3          Pizza Place  0.25
4  American Restaurant  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
               venue  freq
0                Gym  0.08
1  Convenience Store  0.08
2    Supplement Shop  0.08
3     Sandwich Place  0.08
4     Hardware Store  0.08


----New Toronto, Mimico South, Humber Bay Shores----
                  venue  freq
0

In [55]:
#function to sort vanues in descending order 

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
#new dataframe that displays the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Etobicoke_grouped['Neighborhood']

for ind in np.arange(Etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Pizza Place,Skating Rink,Dance Studio,Gym,Sandwich Place,Pub,Pool,Coffee Shop,Pharmacy,Café
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Shopping Plaza,Beer Store,Liquor Store,Cosmetics Shop,Café,Pizza Place,Coffee Shop,Wings Joint,Construction & Landscaping,Fast Food Restaurant
2,"Kingsview Village, St. Phillips, Martin Grove ...",Sandwich Place,Bus Line,Mobile Phone Shop,Pizza Place,Wings Joint,Coffee Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio
3,"Mimico NW, The Queensway West, South of Bloor,...",Wings Joint,Hardware Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Gym
4,"New Toronto, Mimico South, Humber Bay Shores",Café,American Restaurant,Flower Shop,Fried Chicken Joint,Gym,Liquor Store,Mexican Restaurant,Coffee Shop,Pizza Place,Pharmacy


### Cluster Neighborhoods 


In [57]:
# set number of clusters
kclusters = 5

Etobicoke_grouped_clustering = Etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 3, 2, 4, 1], dtype=int32)

In [58]:
#new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Etobicoke_merged = Etobicoke_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Etobicoke_merged = Etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Etobicoke_merged.head() # check the last columns!


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,1.0,Golf Course,Video Store,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Cosmetics Shop,Convenience Store
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,2.0,Shopping Plaza,Beer Store,Liquor Store,Cosmetics Shop,Café,Pizza Place,Coffee Shop,Wings Joint,Construction & Landscaping,Fast Food Restaurant
3,M9P,Etobicoke,Westmount,43.696319,-79.532242,2.0,Pizza Place,Coffee Shop,Discount Store,Intersection,Sandwich Place,Middle Eastern Restaurant,Chinese Restaurant,Fast Food Restaurant,Drugstore,Dance Studio
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,2.0,Sandwich Place,Bus Line,Mobile Phone Shop,Pizza Place,Wings Joint,Coffee Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio


In [65]:
#map the clusters 

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Etobicoke_merged['Latitude'], Etobicoke_merged['Longitude'], Etobicoke_merged['Neighborhood'], Etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='#3186cc',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters


In [66]:
#Cluster 1 

Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 0, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Etobicoke,0.0,Drugstore,Rental Car Location,Wings Joint,Coffee Shop,Flower Shop,Fast Food Restaurant,Discount Store,Dance Studio,Cosmetics Shop,Convenience Store


In [67]:
#Cluster 2

Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 1, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Etobicoke,1.0,Golf Course,Video Store,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Cosmetics Shop,Convenience Store


In [68]:
#Cluster 3 

Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 2, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Etobicoke,2.0,Shopping Plaza,Beer Store,Liquor Store,Cosmetics Shop,Café,Pizza Place,Coffee Shop,Wings Joint,Construction & Landscaping,Fast Food Restaurant
3,Etobicoke,2.0,Pizza Place,Coffee Shop,Discount Store,Intersection,Sandwich Place,Middle Eastern Restaurant,Chinese Restaurant,Fast Food Restaurant,Drugstore,Dance Studio
4,Etobicoke,2.0,Sandwich Place,Bus Line,Mobile Phone Shop,Pizza Place,Wings Joint,Coffee Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio
5,Etobicoke,2.0,Café,American Restaurant,Flower Shop,Fried Chicken Joint,Gym,Liquor Store,Mexican Restaurant,Coffee Shop,Pizza Place,Pharmacy
6,Etobicoke,2.0,Grocery Store,Pharmacy,Fast Food Restaurant,Video Store,Liquor Store,Coffee Shop,Pizza Place,Fried Chicken Joint,Sandwich Place,Beer Store
7,Etobicoke,2.0,Pizza Place,Skating Rink,Dance Studio,Gym,Sandwich Place,Pub,Pool,Coffee Shop,Pharmacy,Café
11,Etobicoke,2.0,Wings Joint,Hardware Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Gym


In [69]:
#Cluster 4

Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 3, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Etobicoke,3.0,Construction & Landscaping,Baseball Field,Breakfast Spot,Wings Joint,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio


In [70]:
#Cluster 5

Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 4, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Etobicoke,4.0,River,Park,Wings Joint,Coffee Shop,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Cosmetics Shop


## The End 